In [5]:
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
import numpy as np 

plt.style.use('fivethirtyeight')

from subprocess import check_output

diabetes.csv



In [6]:
diabetes = pd.read_csv('../data/diabetes.csv')